In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_validate
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold

In [2]:
def generate_dataframe(y_pred, num):
    df = pd.DataFrame(y_pred, columns = ['Predicted'])
    read_file = pd.read_csv('test_II.csv')
    df.insert(0, 'Id', read_file['x'], True)
    df.to_csv(f'submission{num}.csv', index=False)

In [3]:
train_dataset = pd.read_csv('train_feature_desc.csv')
train_dataset.drop('Unnamed: 0', axis=1, inplace=True)
pd.set_option('display.max_rows', None)

train_dataset.fillna(0,inplace=True)
# print(dataset.isnull().sum())

# print(train_dataset.shape)

train_y = train_dataset['Expected']

train_x = train_dataset.drop(['Expected'], axis = 1)
print(train_x.dtypes)
# test data

test_dataset = pd.read_csv('test_feature_desc.csv')
test_dataset.drop('Unnamed: 0', axis=1, inplace=True)

test_dataset.fillna(0, inplace=True)
# print(test_dataset.isnull().sum())

Assay ID                      int64
MaxEStateIndex              float64
MinEStateIndex              float64
MaxAbsEStateIndex           float64
MinAbsEStateIndex           float64
qed                         float64
MolWt                       float64
HeavyAtomMolWt              float64
ExactMolWt                  float64
NumValenceElectrons           int64
NumRadicalElectrons           int64
MaxPartialCharge            float64
MinPartialCharge            float64
MaxAbsPartialCharge         float64
MinAbsPartialCharge         float64
FpDensityMorgan1            float64
FpDensityMorgan2            float64
FpDensityMorgan3            float64
BCUT2D_MWHI                 float64
BCUT2D_MWLOW                float64
BCUT2D_CHGHI                float64
BCUT2D_CHGLO                float64
BCUT2D_LOGPHI               float64
BCUT2D_LOGPLOW              float64
BCUT2D_MRHI                 float64
BCUT2D_MRLOW                float64
BalabanJ                    float64
BertzCT                     

In [4]:
nor_data = train_x / train_x.mean(axis=0)

nor_data.head()

var_thr = VarianceThreshold(threshold=0.8)
var_thr.fit(nor_data)
# var_thr.get_support()


concol = [column for column in train_x.columns if column not in train_x.columns[var_thr.get_support()]]


train_vt = train_x.drop(concol, axis=1)
# print(train_vt.shape)
test_vt = test_dataset.drop(concol, axis=1)
# print(test_vt.shape)

In [7]:
def varthr_xgb24():
    le = LabelEncoder()
    y_train = le.fit_transform(train_y)
    xgb = XGBClassifier(learning_rate=0.1, max_depth=10, n_estimators=1000, objective='binary:logistic', nthread=4, seed=42)
    
    cv = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)
    n_scores = cross_validate(xgb, train_vt, y_train, cv=cv, n_jobs=6, scoring='f1')
    print(np.mean(n_scores['test_score']))

    model = xgb.fit(train_vt, y_train)
    y_pred = model.predict(test_vt)
    y_new = le.inverse_transform(y_pred)
    generate_dataframe(y_new, 24)

In [11]:
if __name__ == '__main__':
    print("Run this function")
#     varthr_xgb24()
    

Run this function
